In [4]:
import pymskt as mskt
import SimpleITK as sitk
import numpy as np
import os

from pymskt.mesh import Mesh

In [10]:
location_seg = '../../data/muscles/segmentation.nii.gz'
location_save = os.path.expanduser('~/Downloads')

In [5]:
# Loading the image in first so we can see what the labels are. 
# This can be skipped and a path provided directly if we know
# the labels of interest. 
seg_image = sitk.ReadImage(location_seg)
array = sitk.GetArrayFromImage(seg_image)
seg_labels = np.unique(array)

mesh1 = Mesh(
    seg_image=seg_image,
    label_idx=seg_labels[1]
    
)

# Below is the command to create the mesh
# all of the defaults inputs parameters are being listed 
# so that the available options are shown. 
mesh1.create_mesh(
    smooth_image=True,            # I suggest leaving this on to help create smooth surfaces.   
    smooth_image_var=0.3125/2,    # this is the variance of a gaussian filter - can probably be bigger for muscles.
    marching_cubes_threshold=0.5, # this probably shouldnt be changed. 
    label_idx=None,               # Can specify this here instead of above if you want. 
    min_n_pixels=None,            # if there is a minimum number of pixels you want for it to create a mesh. 
)

mesh1.resample_surface(clusters=10000) # Resample surface to be a specified # of vertices (w/ in a small margin of error)



Use of `point_arrays` is deprecated. Use `point_data` instead.
Use of `cell_arrays` is deprecated. Use `cell_data` instead.


## Save the mesh

In [11]:
mesh1.save_mesh(os.path.join(location_save, 'mesh1.vtk')) # Save .vtk version of the mesh
mesh1.save_mesh(os.path.join(location_save, 'mesh1.stl')) # Save .stl version of the mesh. 

# If you want to iterate over all of the labels and save meshes: 

### First create a dictionary to store all of the meshes

In [7]:
dict_meshes = {}
for label_idx in seg_labels[1:]:
    dict_meshes[label_idx] = mskt.mesh.Mesh(
        seg_image=seg_image,
        label_idx=label_idx

    )
    
    dict_meshes[label_idx].create_mesh(smooth_image_var=3.0) # I ADDED MORE SMOOTHING FOR THESE
    dict_meshes[label_idx].resample_surface(clusters=10000)

### Save each mesh

In [12]:
for key, mesh in dict_meshes.items():
    mesh.save_mesh(os.path.join(location_save, f'muscle_mesh{key}.vtk')) # Save vtk version
    mesh.save_mesh(os.path.join(location_save, f'muscle_mesh{key}.stl')) # Save stl version

# Below is the viewer that should work to see these in jupyter notebooks
- This wasnt working on my computer for some reason. 

In [13]:
from itkwidgets import view

In [14]:
seg_labels

array([ 0, 14, 16, 18, 20, 22, 24, 26], dtype=uint16)

In [15]:
# view expects geometries to be in a list: 
geometries = [
    dict_meshes[14].mesh  # the Mesh object has the actual mesh inside of it at Mesh.mesh
]
view(geometries=geometries)


Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

### Show multiple meshes at once: 

In [16]:
# view expects geometries to be in a list: 
geometries = [
    dict_meshes[14].mesh,  # the Mesh object has the actual mesh inside of it at Mesh.mesh
    dict_meshes[16].mesh,
    dict_meshes[18].mesh,
    dict_meshes[20].mesh,
    dict_meshes[22].mesh,
    dict_meshes[24].mesh,
    dict_meshes[26].mesh
]
view(geometries=geometries)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…